In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import logging
from helper import getModelDict
delphesDir = os.path.abspath("./DelphesLLP")
os.environ['ROOT_INCLUDE_PATH'] = os.path.join(delphesDir,"external")

import ROOT


ROOT.gSystem.Load(os.path.join(delphesDir,"libDelphes.so"))

ROOT.gInterpreter.Declare('#include "classes/SortableObject.h"')
ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')

c = 3e8

FORMAT = '%(levelname)s: %(message)s at %(asctime)s'
logging.basicConfig(format=FORMAT,datefmt='%m/%d/%Y %I:%M:%S %p')
logger = logging.getLogger()

Welcome to JupyROOT 6.30/06


In [2]:
defaultPars = {'figure.figsize': (5, 4),
               'lines.markersize' : 4,
               'axes.titlesize' : 13,
               'font.size' : 13,
               'axes.labelsize' : 16,
               'xtick.labelsize' : 13,
               'ytick.labelsize' : 13,
               'legend.fontsize' : 10,
               "text.usetex": True,
               "font.family": "sans-serif",
               "font.sans-serif": ["Helvetica"],
               'font.family':'Times New Roman', 
               'font.serif':'Times New Roman',
               "savefig.dpi" : 300,
               'contour.linewidth' : 2.0,
               'lines.linewidth' : 2.0,
               'axes.grid' : True,
               'grid.linewidth' : 0.5,
               'grid.color' : 'lightgray',
               'axes.axisbelow' : True
               }
plt.rcParams.update(defaultPars)

### Files to be compared

In [3]:
filesDict = {
                r'Signal' : './pp2chi0chi0J_scan/Events/run_01/ddmH_mS_500_m1_244_dm_90_delphes_events.root',
                # r'Signal (no smearing)' : './pp2chi0chi0J_scan/Events/run_01/ddmH_mS_500_m1_244_dm_90_delphes_events_nosmear.root',
}

modelDict = {}
for key,f in list(filesDict.items()):
    if not os.path.isfile(f):
        print(f'File {f} not found. Skipping.')
        filesDict.pop(key)
    else:
        modelDict[key] = getModelDict(f,verbose=False)

if len(set([tuple(d.items()) for d in modelDict.values()])) > 1:
    print('Several models being used!')
    mDict = None
else:
    mDict = list(modelDict.values())[0]


### Get kinematical variables for each file

In [ ]:
L1metStr = r'$E_T^{\rm miss}$ (Hardware Trigger, N-1) (GeV)'
L1njStr = r'$n_{j}$ (Hardware Trigger, N)'
L1pTj1Str = r'Leading jet $E_{T}$ (Hardware Trigger, N) (GeV)'
L1dPhi = r'$\Delta \phi^{min} (MET,j)$ (Hardware Trigger)'
L1metPartonStr = r'$E_T^{\rm miss}$ (Parton Level, N-1) (GeV)'

cols = ['label',L1metStr,L1njStr,L1pTj1Str,L1dPhi,L1metPartonStr]


dataList = []
for label,file in filesDict.items():   
    f = ROOT.TFile(file,'read')
    tree = f.Get("Delphes")
    nevts = tree.GetEntries()
    for ievt in range(nevts):
        tree.GetEntry(ievt)

        # Get parton level MET
        llps = list(tree.llpParticles)
        invisibles = [p for p in tree.llpDirectDaughters 
                      if abs(p.PID) == 4000022]
        invisibles = sorted(invisibles, key = lambda p: p.M1)
        pInvTot = np.zeros(3)
        for illp,llp in enumerate(llps):
            daughter = invisibles[illp]
            decayTime = daughter.T
            if decayTime < 15e-9: # if LLP decays on-time, add its daughter momentum
                pInv = np.array([daughter.Px,daughter.Py,
                                 daughter.Pz])
            else: # add the LLP momentum
                pInv = np.array([llp.Px,llp.Py,
                                llp.Pz])
            pInvTot += pInv
        
        metParton = np.linalg.norm(pInvTot[:2])




        metOnTime = tree.L1METOnTime.At(0)
        jetsDelayed = list(tree.L1JetDelayed)
        jetsDelayed = sorted(jetsDelayed, 
                         key = lambda j: j.PT, reverse=True)

        met = metOnTime.MET
        nj = len(jetsDelayed)
        if nj > 0:
            pTj1 = jetsDelayed[0].PT
            dphi_min = 10000.0
            for j in jetsDelayed[:6]:
                dphi = np.abs(j.Phi-metOnTime.Phi)
                if dphi > np.pi:
                    dphi = 2*np.pi-dphi
                dphi_min = min(dphi,dphi_min)
        else:
            pTj1 = 0.0
            dphi_min = 5.0

        
        
        dataList.append([label,met,nj,pTj1,dphi_min,metParton])
    f.Close()
df = pd.DataFrame(columns=cols,data=dataList)
        

### Select Event

In [5]:
df[df[L1metStr]/df[L1metPartonStr] < 0.01]

,label,"$E_T^{\rm miss}$ (Hardware Trigger, N-1) (GeV)","$n_{j}$ (Hardware Trigger, N)","Leading jet $E_{T}$ (Hardware Trigger, N) (GeV)","$\Delta \phi^{min} (MET,j)$ (Hardware Trigger)","$E_T^{\rm miss}$ (Parton Level, N-1) (GeV)"
8125,Signal,3.524345,0,0.0,5.0,420.548044
8629,Signal,2.017742,0,0.0,5.0,369.624300
13048,Signal,0.258596,0,0.0,5.0,29.657803
13206,Signal,0.472844,0,0.0,5.0,65.549550
15973,Signal,1.722918,0,0.0,5.0,202.122705
17176,Signal,1.366636,0,0.0,5.0,283.398427
21929,Signal,4.870231,0,0.0,5.0,631.647652
24879,Signal,0.392351,0,0.0,5.0,52.147224
25816,Signal,0.983118,0,0.0,5.0,158.365862
28359,Signal,0.211624,0,0.0,5.0,48.551797


### Compare Parton Level and Calo MET

In [6]:
ievt = 57
f = ROOT.TFile(file,'read')
tree = f.Get("Delphes")
tree.GetEntry(ievt)


# Get parton level MET
llps = list(tree.llpParticles)
invisibles = [p for p in tree.llpDirectDaughters 
                if abs(p.PID) == 4000022]
visibles =  [p for p in tree.llpDirectDaughters 
                if abs(p.PID) != 4000022]
invisibles = sorted(invisibles, key = lambda p: p.M1)
pInvTot = np.zeros(4)
pISR = np.zeros(4)
for illp,llp in enumerate(llps):
    pLLP = np.array([llp.Px,llp.Py,
                        llp.Pz,llp.E])
    pISR -= pLLP
    daughter = invisibles[illp]
    decayTime = daughter.T
    print(f'PID = {llp.PID}, t_decay (ns) = {decayTime*1e9:1.2f}')
    if decayTime < 10e-9: # if LLP decays on-time, add its daughter momentum
        pInv = np.array([daughter.Px,daughter.Py,
                            daughter.Pz,daughter.E])
    else: # add the LLP momentum
        pInv = pLLP
    print(f'  pLLP = {pLLP}')
    print(f'  pInv = {pInv}')
    pInvTot += pInv
pISR[3] = np.linalg.norm(pISR[:3])
metParton = np.linalg.norm(pInvTot[:2])

metOnTime = tree.L1METOnTime.At(0)


print(f'Parton MET = {metParton:1.2f}, Calo MET = {metOnTime.MET:1.2f}')
phi = np.arctan2(pInvTot[1],pInvTot[0])
print(f'Parton Phi = {phi:1.2f}, Calo Phi = {metOnTime.Phi:1.2f}')
eta = np.arccosh(pInvTot[3]/metParton)
print(f'Parton Eta = {eta:1.2f}, Calo Eta = {metOnTime.Eta:1.2f}')

print('\nVisibles (Parton Level, N-1):')
pTot = pISR[:]
pT = np.linalg.norm(pISR[:2])
E = pISR[3]
phi = np.arctan2(pISR[1],pISR[0])
eta = np.arccosh(E/pT)    
print(f'  ISR: E = {E:1.2f}, eta = {eta:1.2f}, phi = {phi:1.2f}')
for p in visibles:
    if p.T > 10e-9:
        continue
    p4 = np.array([p.Px,p.Py,p.Pz,p.E])
    pTot += p4
    pT = np.linalg.norm(p4[:2])
    E = p4[3]
    phi = np.arctan2(p4[1],p4[0])
    eta = np.arccosh(E/pT)    
    print(f'  {p.PID}: E = {E:1.2f}, eta = {eta:1.2f}, phi = {phi:1.2f}')
print(f'MET = {np.linalg.norm(-pTot[:2])}')

f.Close()

PID = 4000023, t_decay (ns) = 0.29
  pLLP = [-137.89854431  489.68240356 -256.28048706  619.69396973]
  pInv = [-133.1925354   380.86618042 -240.64955139  494.39611816]
PID = -4000023, t_decay (ns) = 41.34
  pLLP = [-129.27388     405.93423462 -125.77534485  506.8034668 ]
  pInv = [-129.27388     405.93423462 -125.77534485  506.8034668 ]
Parton MET = 829.42, Calo MET = 19.22
Parton Phi = 1.89, Calo Phi = 3.07
Parton Eta = 0.63, Calo Eta = -2.26

Visibles (Parton Level, N-1):
  ISR: E = 1009.69, eta = 0.40, phi = -1.28
  5: E = 70.28, eta = 0.25, phi = 1.40
  -5: E = 55.01, eta = 0.67, phi = 1.94
MET = 829.423612714599


### Check Calo Towers

In [7]:
f = ROOT.TFile(file,'read')
tree = f.Get("Delphes")
tree.GetEntry(ievt)

pTot = np.zeros(3)
for tower in tree.L1TowerOnTime:
    ET,phi,eta = tower.ET,tower.Phi,tower.Eta
    pTower = np.array([ET*np.cos(phi),ET*np.sin(phi),ET*np.sinh(eta)])
    pTot += pTower
    print(f'(eta,phi) = ({tower.Eta:1.2f},{tower.Phi:1.2}), t= {tower.T:1.2e}')
print('MET = ',np.linalg.norm(-pTot[:2]))


f.Close()

(eta,phi) = (-3.85,-2.5), t= 1.24e-08
(eta,phi) = (-3.85,-2.1), t= 1.24e-08
(eta,phi) = (-3.85,-1.3), t= 1.24e-08
(eta,phi) = (-3.85,2.2), t= 1.24e-08
(eta,phi) = (-3.85,2.8), t= 1.24e-08
(eta,phi) = (-3.75,2.2), t= 1.24e-08
(eta,phi) = (-3.75,3.1), t= 1.24e-08
(eta,phi) = (-3.65,-3.1), t= 1.24e-08
(eta,phi) = (-3.65,1.9), t= 1.24e-08
(eta,phi) = (-3.65,2.3), t= 1.24e-08
(eta,phi) = (-3.55,0.65), t= 1.24e-08
(eta,phi) = (-3.25,-0.22), t= 1.24e-08
(eta,phi) = (-3.25,2.7), t= 1.24e-08
(eta,phi) = (-3.15,-1.6), t= 1.24e-08
(eta,phi) = (-3.15,0.044), t= 1.25e-08
(eta,phi) = (-3.15,0.48), t= 1.24e-08
(eta,phi) = (-3.15,1.6), t= 1.24e-08
(eta,phi) = (-3.15,2.6), t= 1.24e-08
(eta,phi) = (-2.95,0.48), t= 1.24e-08
(eta,phi) = (-2.95,2.1), t= 1.24e-08
(eta,phi) = (-2.95,2.8), t= 1.24e-08
(eta,phi) = (-2.85,-0.92), t= 1.24e-08
(eta,phi) = (-2.85,0.48), t= 1.24e-08
(eta,phi) = (-2.75,-2.2), t= 1.24e-08
(eta,phi) = (-2.75,-0.48), t= 1.24e-08
(eta,phi) = (-2.75,1.0), t= 1.24e-08
(eta,phi) = (-2.65,0